In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

data = pd.read_csv("output_file.csv")

# Drop unnecessary columns
data.drop(['Date', 'Time'], axis=1, inplace=True)

# Separate features and target variable
X = data.drop('Radiation', axis=1)
y = data['Radiation']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [2]:
from sklearn.neural_network import MLPRegressor

def create_ann():
    # Define the neural network structure
    ann = MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=200, random_state=42)
    return ann


In [3]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.exceptions import ConvergenceWarning
from sklearn.neural_network import MLPRegressor
import warnings

# Suppress convergence warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Define OIO parameters
num_particles = 30
max_iterations = 100
c1 = 0.1  # Acceleration constant
c2 = 0.1  # Damping constant
c3 = 0.1  # Step size constant

# Initialize particles
# Each particle will represent [num_neurons_layer1, num_neurons_layer2, learning_rate]
particles = np.random.uniform(0, 1, (num_particles, 3))
particles[:, :2] = particles[:, :2] * 100  # Number of neurons in the range [0, 100]

# Define the fitness function
def fitness_function(particle):
    num_neurons_layer1 = int(particle[0])
    num_neurons_layer2 = int(particle[1])
    learning_rate = particle[2]
    
    ann = MLPRegressor(hidden_layer_sizes=(num_neurons_layer1, num_neurons_layer2),
                       learning_rate_init=learning_rate, max_iter=200, random_state=42)
    ann.fit(X_train, y_train)
    predictions = ann.predict(X_test)
    mse = np.mean((predictions - y_test) ** 2)
    return mse

# Store MSE values at each iteration
mse_progress = []

# Main OIO loop with progress bar
for iteration in tqdm(range(max_iterations), desc="OIO Training"):
    fitness_values = []
    for i in range(num_particles):
        fitness = fitness_function(particles[i])
        fitness_values.append(fitness)
        
        # Randomly generate a search direction
        search_direction = np.random.uniform(-1, 1, particles.shape[1])
        
        # Update particle position
        particles[i] = particles[i] + c1 * search_direction * fitness / (1 + c2 * np.linalg.norm(particles[i]))
        
        # Bound the particle position
        particles[i, :2] = np.clip(particles[i, :2], 1, 100)  # Neurons range [1, 100]
        particles[i, 2] = np.clip(particles[i, 2], 0.0001, 1)  # Learning rate range [0.0001, 1]
    
    # Store the minimum fitness value for the current iteration
    mse_progress.append(min(fitness_values))
    
    # Debugging: Print iteration and current best MSE
    print(f"Iteration {iteration + 1}/{max_iterations}, Best MSE: {mse_progress[-1]}")

# Plot the MSE progress
plt.figure(figsize=(10, 6))
plt.plot(range(max_iterations), mse_progress, marker='o')
plt.title('OIO Training Progress')
plt.xlabel('Iteration')
plt.ylabel('Mean Squared Error (MSE)')
plt.grid(True)
plt.show()

# Select the best particle
best_particle_idx = np.argmin(fitness_values)
best_particle = particles[best_particle_idx]

# Train the final ANN with the best parameters
num_neurons_layer1 = int(best_particle[0])
num_neurons_layer2 = int(best_particle[1])
learning_rate = best_particle[2]

final_ann = MLPRegressor(hidden_layer_sizes=(num_neurons_layer1, num_neurons_layer2),
                         learning_rate_init=learning_rate, max_iter=200, random_state=42)
final_ann.fit(X_train, y_train)

# Evaluate the final model
final_predictions = final_ann.predict(X_test)
final_mse = np.mean((final_predictions - y_test) ** 2)
print("Final MSE:", final_mse)


OIO Training:   1%|          | 1/100 [00:41<1:08:54, 41.76s/it]

Iteration 1/100, Best MSE: 29930.53422448422


OIO Training:   2%|▏         | 2/100 [03:14<2:55:11, 107.26s/it]

Iteration 2/100, Best MSE: 33454.59817000285
